---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [13]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [14]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### Question 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [15]:
def answer_one():
    
    
    return (len(spam_data[spam_data["target"]==1])/len(spam_data)*100)

In [5]:
answer_one()

13.406317300789663

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [16]:
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():
    vectorizer = CountVectorizer().fit(X_train)
    vocab = vectorizer.vocabulary_
    tokens = {token:len(token) for token in list(vocab.keys())}
    
    return max(tokens,key=tokens.get)

In [9]:
answer_two()

'com1win150ppmx3age16subscription'

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [17]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():
    
    countvect = CountVectorizer().fit(X_train)
    transformed = countvect.transform(X_train)
    model = MultinomialNB(0.1)
    model.fit(transformed,y_train)
    predictions = model.predict(countvect.transform(X_test))
    
    return roc_auc_score(y_test, predictions)

In [11]:
answer_three()

0.97208121827411165

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_four():
    vect = TfidfVectorizer().fit(X_train)
    features = np.array(vect.get_feature_names())
    sorted_tfidf_index = vect.transform(X_train).max(0).toarray()[0].argsort()

    
    return (pd.Series(features[sorted_tfidf_index[:20]]),pd.Series(features[sorted_tfidf_index[:-21:-1]]))

In [14]:
answer_four()

(0      sympathetic
 1           healer
 2           aaniye
 3       dependable
 4        companion
 5         listener
 6         athletic
 7     exterminator
 8     psychiatrist
 9             pest
 10      determined
 11            chef
 12      courageous
 13         stylist
 14    psychologist
 15       organizer
 16         pudunga
 17          venaam
 18          diwali
 19        mornings
 dtype: object, 0     146tf150p
 1        havent
 2          home
 3          okie
 4         thanx
 5            er
 6      anything
 7           lei
 8          nite
 9           yup
 10        thank
 11           ok
 12        where
 13      beerage
 14      anytime
 15          too
 16         done
 17          645
 18         tick
 19        blank
 dtype: object)

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [19]:
def answer_five():
    
    vect = TfidfVectorizer(min_df=3).fit(X_train)
    transformed = vect.transform(X_train)
    model = MultinomialNB(0.1)
    model.fit(transformed,y_train)
    predictions = model.predict(vect.transform(X_test))
    
    return roc_auc_score(y_test, predictions)
    

In [16]:
answer_five()

0.94162436548223349

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [20]:
def answer_six():
    
    spam_docs = spam_data[spam_data['target']==1]['text']
    nonspam_docs = spam_data[spam_data['target']==0]['text']

    # sum of no. of characters in each doc
    spam_len = spam_docs.apply(len).sum()
    nonspam_len = nonspam_docs.apply(len).sum()
    
    return (nonspam_len/len(nonspam_docs),spam_len/len(spam_docs))

In [18]:
answer_six()

(71.023626943005183, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [21]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [22]:
from sklearn.svm import SVC

def answer_seven():
    
    vect = TfidfVectorizer(min_df=5).fit(X_train)
    transformed_train = vect.transform(X_train)
    transformed_test= vect.transform(X_test)
    
    len_train = [len(x) for x in X_train]
    len_test = [len(x) for x in X_test]
    
    transformed_train = add_feature(transformed_train, len_train)
    transformed_test = add_feature(transformed_test, len_test)
    
    clf = SVC(C=10000)
    clf.fit(transformed_train, y_train)
    predictions = clf.predict(transformed_test)
    
    return roc_auc_score(y_test, predictions)
 

In [23]:
answer_seven()

0.95813668234215565

### Question 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [23]:
def answer_eight():
    
    spam_docs = spam_data[spam_data['target']==1]['text']
    nonspam_docs = spam_data[spam_data['target']==0]['text']
    
    #lists of length of docs
    spam_len = [len(doc) for doc in spam_docs]
    nonspam_len = [len(doc) for doc in nonspam_docs]
    
    #lists of #digits in docs
    spam_digits = [sum(x.isnumeric() for x in doc) for doc in spam_docs]
    nonspam_digits = [sum(x.isnumeric() for x in doc) for doc in nonspam_docs]
    
    return (np.mean(nonspam_digits), np.mean(spam_digits))

In [38]:
answer_eight()

(0.29927461139896372, 15.76037483266399)

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [5]:
from sklearn.linear_model import LogisticRegression

def answer_nine():
    
    vect = TfidfVectorizer(min_df=5, ngram_range=(1,3)).fit(X_train)
    transformed_train = vect.transform(X_train)
    transformed_test= vect.transform(X_test)
    
    dig_train = [sum(char.isnumeric() for char in x) for x in X_train]
    dig_test = [sum(char.isnumeric() for char in x) for x in X_test]
    
    transformed_train = add_feature(transformed_train, dig_train)
    transformed_test = add_feature(transformed_test, dig_test)
    
    clf = LogisticRegression(C=100)
    clf.fit(transformed_train, y_train)
    predictions = clf.predict(transformed_test)
    
    return roc_auc_score(y_test, predictions)


In [12]:
answer_nine()

0.96787090640544626

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [32]:
import re

def answer_ten():
    
    spam_docs = spam_data[spam_data['target']==1]['text']
    nonspam_docs = spam_data[spam_data['target']==0]['text']
    
    nonword_nonspam = [len(re.findall("\W",doc)) for doc in nonspam_docs]
    nonword_spam = [len(re.findall("\W",doc)) for doc in spam_docs]
    
    
    return (sum(nonword_nonspam)/len(nonspam_docs),sum(nonword_spam)/len(spam_docs) )

In [33]:
answer_ten()

(17.29181347150259, 29.041499330655956)

### Question 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [34]:
def answer_eleven():
    
    len_train = [len(x) for x in X_train]
    len_test = [len(x) for x in X_test]
    dig_train = [sum(char.isnumeric() for char in x) for x in X_train]
    dig_test = [sum(char.isnumeric() for char in x) for x in X_test]
    
    
    nan_train = X_train.str.count('\W')
    nan_test = X_test.str.count('\W')
    
    cv = CountVectorizer(min_df = 5, ngram_range=(2,5), analyzer='char_wb').fit(X_train)
    X_train_cv = cv.transform(X_train)
    X_test_cv = cv.transform(X_test)
    
    X_train_cv = add_feature(X_train_cv, [len_train, dig_train, nan_train])
    X_test_cv = add_feature(X_test_cv, [len_test, dig_test, nan_test])
    
    clf = LogisticRegression(C=100).fit(X_train_cv, y_train)
    pred = clf.predict(X_test_cv)
    
    score = roc_auc_score(y_test, pred)
    
    feature_names = np.array(cv.get_feature_names() + ['length_of_doc', 'digit_count', 'non_word_char_count'])
    sorted_coef_index = clf.coef_[0].argsort()
    small_coeffs = list(feature_names[sorted_coef_index[:10]])
    large_coeffs = list(feature_names[sorted_coef_index[:-11:-1]])
    
    return (score, small_coeffs, large_coeffs)
    

In [35]:
answer_eleven()

(0.97885931107074342,
 ['. ', '..', '? ', ' i', ' y', ' go', ':)', ' h', 'go', ' m'],
 ['digit_count', 'ne', 'ia', 'co', 'xt', ' ch', 'mob', ' x', 'ww', 'ar'])